In [21]:
from astropy.io import fits
import os
import numpy as np
from scipy.interpolate import InterpolatedUnivariateSpline, RegularGridInterpolator


"""
the path_to_grid variable must be set to the filepath for the directory in which
the grids are stored such that the exposure time functions can find these files

ex: path_to_grid = '/home/anaconda3/lib/python3.6/site-packages/neid_calculator/'
"""

path_to_grid=os.getcwd()


In [22]:
def NEID_exptime_SNR(teff, vmag, snr, wavelength):
    """
    calculate exposure time required to achieve specified SNR for given inputs
    teff:          Effective Temperature (K)
    vmag:          V-band magnitude
    snr:           Desired SNR
    wavelength:    wavelength (nm) at which exposure time should be calculated
    """
    
    exptime_grid = fits.open(path_to_grid+'photon_grid_exptime.fits')[0].data
    teff_grid = fits.open(path_to_grid+'photon_grid_teff.fits')[0].data
    vmag_grid = fits.open(path_to_grid+'photon_grid_vmag.fits')[0].data
    logexp= np.log10(exptime_grid)
    
    bound_test=True
    if teff<np.min(teff_grid) or teff>np.max(teff_grid):
        print("Temperature out of bounds. The allowed range is %d K to %d K." % (np.min(teff_grid), np.max(teff_grid)))
        bound_test=False
    if vmag<np.min(vmag_grid) or vmag>np.max(vmag_grid):
        print("Magnitude out of bounds. The allowed range is V = %d to V = %d." % (np.min(vmag_grid), np.max(vmag_grid)))
        bound_test=False
    if bound_test==False:
        return np.nan
    
    wavelength_grid = fits.open(path_to_grid+'order_wvl_centers.fits')[0].data[1]
    order_loc=np.where(np.abs(wavelength_grid-wavelength)<0.1)[0][0]
    snr_grid_order=fits.open(path_to_grid+'snr_master_order.fits')[0].data
    snr_grid=snr_grid_order[order_loc]
        
    
    teff_index=InterpolatedUnivariateSpline(teff_grid, 
                            np.arange(len(teff_grid), dtype=np.double))(teff)

    vmag_index=InterpolatedUnivariateSpline(vmag_grid, 
                            np.arange(len(vmag_grid), dtype=np.double))(vmag)  
    j=0
    eta=0
    while eta<snr:
        exptime=2*(j+6)
        if exptime>np.max(exptime_grid):
        #     print("\nMaximum Exposure Time Exceeded (t>3600s).\n")
            return np.nan
        exptime_index=InterpolatedUnivariateSpline(logexp, np.arange(len(exptime_grid),
                                                dtype=np.double))(np.log10(exptime))
        snr_interpolator=RegularGridInterpolator((np.arange(len(exptime_grid)),
                              np.arange(len(vmag_grid)),
                              np.arange(len(teff_grid))),
                                    snr_grid)
        inputs=[exptime_index, vmag_index, teff_index]
        eta=snr_interpolator(inputs)[0]
        j+=1
    return exptime

In [23]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Label
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "CMU Serif",
    "font.size": 12,
    "figure.dpi": 150
}) 
loading_label = Label(value="Loading...")

In [ ]:


def plot_exposure_vs_magnitude(Teff, SNR):
    
    loading_label.value = "Loading..."
    display(loading_label)
    
    magnitudes = np.linspace(3, 17, 100)
    exposure_times = [NEID_exptime_SNR(vmag=mag, teff=Teff, snr=SNR,wavelength=573.6) for mag in magnitudes]
    
    fig, ax = plt.subplots(figsize=(7, 4))
    fig.patch.set_facecolor('#1e1e1e')  # Dark gray outside area
    ax.set_facecolor('#2e2e2e')  # Slightly lighter gray for the plot background
    
    # Set dark grid lines, ticks, and text
    ax.plot(magnitudes, exposure_times, color='cyan',label='Exposure Function')
    ax.hlines(y=180,xmin=np.min(magnitudes),xmax=np.max(magnitudes),color='red',linestyle='--',label='180 s')
    ax.set_yscale('log')
    ax.set_xlabel('Magnitude', color='white')
    ax.set_ylabel('Exposure Time (s)', color='white')
    ax.set_title('NEID Exposure Time Calculator', color='white')
    ax.grid(True, color='#444444')  # Dark grid lines
    ax.legend(loc='upper left')
    
    # Customize tick color
    ax.tick_params(colors='white')

# Add interactivity for effective temperature, SNR, seeing, and mode selection
interact(
    plot_exposure_vs_magnitude,
    Teff=FloatSlider(min=2700, max=6600, step=50, value=5000,description='T (K)'),
    SNR=FloatSlider(min=10, max=625, step=10, value=100)
)